# 02 · Feature Engineering e Pipelines
## Detecção de Lavagem de Dinheiro (AML)

**Objetivo:** Construir features robustas para modelagem, com pipelines reprodutíveis e foco em prevenção de data leakage.

### Introdução

Em detecção de AML, features temporais e de rede são cruciais porque transações suspeitas frequentemente envolvem padrões sequenciais e conexões entre entidades. Escolhi implementar agregações por entidade porque dados financeiros são intrinsecamente temporais - um cliente que movimenta grandes valores em janelas curtas pode indicar comportamento de lavagem. Para evitar data leakage, todas as features são calculadas apenas com dados históricos disponíveis no momento da transação.

### Pipeline de Transformação

```
Raw Data → Cleaning → Feature Engineering → Validation → Pipeline
    ↓         ↓             ↓                ↓          ↓
- Load     - Remove     - Temporal       - Correlation - Sklearn
- Parse    - Duplicates - Aggregations   - Analysis   - Pipeline
- Anonymize - Missing   - Network        - Leakage    - Reproducible
            - Values    - Features       - Check      - Training
```

### Estratégia de Features

1. **Features Temporais**: Agregações por entidade em janelas móveis (7d, 30d)
2. **Features de Rede**: Métricas de conectividade entre contas
3. **Features Categóricas**: Encoding seguro sem leakage
4. **Features Derivadas**: Razões e taxas calculadas eticamente

> **Decisão de Design:** Priorizei features interpretáveis sobre complexas para garantir que o modelo possa ser explicado para compliance regulatória.

In [1]:
# Setup e Importações
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Adicionar src ao path
sys.path.append(str(Path('..').resolve()))

# Imports core
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# Configuração visual
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

In [2]:
# Configurar sys.path para importações
import sys
from pathlib import Path
import importlib

# Adicionar diretório src ao path se não estiver lá
sys.path.append(str(Path('..').resolve()))

# Force reload do módulo
if 'src.features.aml_features' in sys.modules:
    importlib.reload(sys.modules['src.features.aml_features'])

# Importações das funções refatoradas
from src.features.aml_features import (
    load_raw_transactions,
    validate_data_compliance,
    clean_transactions,
    impute_and_encode,
    aggregate_by_entity,
    compute_network_features,
    create_temporal_features,
    create_network_features,
    encode_categorical_features,

AMLFeaturePipeline
)

# Importar Pattern Feature Engineer
from src.features.pattern_engineering import PatternFeatureEngineer

Funções locais implementadas com sucesso!


## ▸ Carregamento e Preparação dos Dados

Carregamos os dados brutos e aplicamos limpeza inicial, garantindo compliance e anonimização.

In [3]:
# Carregar dados brutos usando função refatorada
# Você pode especificar o caminho completo do arquivo desejado
data_file_path = '../data/processed/transactions_enriched_final.csv'  # <- ALTERE ESTE CAMINHO CONFORME NECESSÁRIO
df_raw = load_raw_transactions(data_path=data_file_path)

# Validar compliance usando função refatorada
is_compliant = validate_data_compliance(df_raw)
print(f"Compliance check: {'Passou' if is_compliant else 'Falhou'}")

# Estatísticas iniciais
print(f"Dataset carregado: {len(df_raw):,} transações")
print(f"Período: {df_raw['timestamp'].min()} até {df_raw['timestamp'].max()}")
print(f"Taxa de fraude: {df_raw['is_fraud'].mean():.3%}")

# Preview dos dados
df_raw.head()

 Tentando carregar de: ../data/processed/transactions_enriched_final.csv
 Caminho resolvido: C:\Users\gafeb\AML_project\data\processed\transactions_enriched_final.csv
 É arquivo? True
 Existe? True
 Carregado arquivo específico: transactions_enriched_final.csv
 Aplicando mapeamento para arquivo processado: transactions_enriched_final.csv
 Colunas antes do mapeamento: ['Timestamp', 'From Bank ID', 'From Account', 'To Bank ID', 'To Account']...
 Colunas após mapeamento: ['timestamp', 'From Bank ID', 'source', 'To Bank ID', 'target']...
 Arquivo processado mapeado: ['timestamp', 'From Bank ID', 'source', 'To Bank ID', 'target']...
Compliance check: Passou
Dataset carregado: 5,078,345 transações
Período: 2022-09-01 00:00:00 até 2022-09-18 16:18:00
Taxa de fraude: 0.102%


,timestamp,From Bank ID,source,To Bank ID,target,Amount Received,Receiving Currency,amount,Payment Currency,payment_format,...,hour,date,is_night_transaction,same_bank_transaction,same_entity_transaction,high_amount_risk,from_sole_proprietorship_flag,from_partnership_flag,from_corporation_flag,from_individual_flag
0,2022-09-01 00:20:00,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,...,0,2022-09-01,False,True,True,False,False,True,False,False
1,2022-09-01 00:20:00,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,...,0,2022-09-01,False,False,False,False,False,True,False,False
2,2022-09-01 00:00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,...,0,2022-09-01,False,True,True,False,False,True,False,False
3,2022-09-01 00:02:00,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,...,0,2022-09-01,False,True,True,False,True,False,False,False
4,2022-09-01 00:06:00,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,...,0,2022-09-01,False,True,True,False,False,True,False,False


## ▸ Limpeza e Pré-processamento

Aplicamos limpeza para remover duplicatas, valores inválidos e garantir integridade dos dados.

In [4]:
# Limpeza dos dados usando função refatorada
df_clean = clean_transactions(df_raw)

# Estatísticas pós-limpeza
duplicates_removed = len(df_raw) - len(df_clean)
print(f"Duplicatas removidas: {duplicates_removed}")
print(f"Dataset limpo: {len(df_clean):,} transações")

# Distribuição da variável target
fraud_dist = df_clean['is_fraud'].value_counts(normalize=True)
print(f"Distribuição da classe: Normal {fraud_dist[0]:.3%}, Fraudulenta {fraud_dist[1]:.3%}")

 Aplicando limpeza de dados...
 Limpeza concluída: 9 duplicatas removidas
Duplicatas removidas: 9
Dataset limpo: 5,078,336 transações
Distribuição da classe: Normal 99.898%, Fraudulenta 0.102%


## ▸ Features Temporais

Criamos agregações por entidade em janelas temporais para capturar padrões comportamentais. Esta é a feature mais importante porque transações de lavagem frequentemente ocorrem em bursts temporais.

In [5]:
# Force reload of the module to get latest changes
import importlib
import src.features.aml_features
importlib.reload(src.features.aml_features)
from src.features.aml_features import create_temporal_features

# Features temporais usando função refatorada - TESTE COM SAMPLE PEQUENO
print(f"Dataset original: {df_clean.shape[0]:,} linhas")
df_sample = df_clean.sample(n=min(50000, len(df_clean)), random_state=42)  # Sample menor para teste
print(f"Usando sample de {df_sample.shape[0]:,} linhas para teste")

temporal_features_df = create_temporal_features(df_sample, windows=[7, 30])

print(f"Features temporais criadas: {temporal_features_df.shape[1]} colunas")

Funções locais implementadas com sucesso!
Dataset original: 5,078,336 linhas
Usando sample de 50,000 linhas para teste


2025-10-18 15:09:12,195 - INFO - Creating temporal features with pandas...


[temporal] Starting temporal feature creation with pandas...
[temporal] Processing window=7 days...
[temporal] Finished window=7d in 174.91s
[temporal] Processing window=30 days...
[temporal] Finished window=30d in 181.63s


2025-10-18 15:15:08,831 - INFO - Created 12 temporal features using pandas


[temporal] Done. Total time: 356.63s
Features temporais criadas: 42 colunas


In [6]:
# Mostrar apenas as colunas temporais criadas
temporal_cols = [col for col in temporal_features_df.columns if col.startswith('source_amount_') or col.startswith('hour') or col.startswith('day_of_week') or col.startswith('is_business_hours') or col.startswith('is_weekend')]
print(f"Criadas {len(temporal_cols)} features temporais. Ex: {sorted(temporal_cols)[:3]}...")

Criadas 10 features temporais. Ex: ['day_of_week', 'hour', 'is_business_hours']...


In [18]:
# Carregar dataset com features de patterns para uso posterior
import pandas as pd
from pathlib import Path

# Caminho do arquivo salvo
data_dir = Path('../data/processed/')
features_file = data_dir / 'features_with_patterns.pkl'

# Carregar dataset
df = pd.read_pickle(features_file)

In [16]:
from src.features.iv_calculator import calculate_iv, interpret_iv, get_predictive_features
# Calcular IV para features temporais
# Primeiro criar df_final com features temporais
temporal_cols = [col for col in temporal_features_df.columns if col.startswith('source_amount_') or col.startswith('hour') or col.startswith('day_of_week') or col.startswith('is_business_hours') or col.startswith('is_weekend')]
df_with_temporal = df.copy()

for col in temporal_cols:
    df_with_temporal[col] = temporal_features_df[col]

iv_results_temporal = calculate_iv(
    df_with_temporal,
    target_col='is_fraud',
    bins=10,
    max_iv=10.0,
    min_samples=1,
    max_unique_values=100000
)

temporal_iv_results = iv_results_temporal[iv_results_temporal['variable'].isin(temporal_cols)].copy()

temporal_iv_results.to_csv('../artifacts/temporal_features_iv_report.csv', index=False)
print("Relatório de IV salvo em artifacts.")

predictive_count = len(temporal_iv_results[temporal_iv_results['IV'] >= 0.02])
if predictive_count == 0:
    print("DECISÃO: EXCLUIR todas as features temporais")
else:
    print(f"DECISÃO: Usar {predictive_count} features temporais com IV >= 0.02")

Relatório de IV salvo em artifacts.
DECISÃO: EXCLUIR todas as features temporais


## ▸ Features de Rede

Analisamos a conectividade entre contas para identificar padrões de lavagem estruturada. Contas fraudulentas frequentemente formam clusters densos.

In [10]:
# Usar função importada para criar features de rede
network_features_df = create_network_features(df_clean)

print(f"Features de rede criadas para {len(network_features_df)} nós")
print(f"Estatísticas da rede: Grau médio {network_features_df['degree'].mean():.2f}, Centralidade média {network_features_df['degree_centrality'].mean():.4f}")

# Visualizar distribuição dos graus (usando apenas uma amostra para performance)
network_features_df.head()

2025-10-18 15:18:24,112 - INFO - Creating network features...
2025-10-18 15:18:47,583 - INFO - Created network features for 515080 nodes


Features de rede criadas para 515080 nós
Estatísticas da rede: Grau médio 3.94, Centralidade média 0.0000


,node,degree_centrality,in_degree_centrality,out_degree_centrality,degree,in_degree,out_degree
0,8000EBD30,0.000029,0.000025,0.000004,15,13,2
1,8000F4580,0.000004,0.000000,0.000004,2,0,2
2,8000F5340,0.000035,0.000029,0.000006,18,15,3
3,8000F4670,0.000004,0.000002,0.000002,2,1,1
4,8000F5030,0.000043,0.000039,0.000004,22,20,2


In [11]:
# Preparar dados para análise IV das features de rede
from src.features.iv_calculator import calculate_iv, interpret_iv, get_predictive_features

# Combinar features de rede com dados transacionais
df_with_network = df.copy()

# Merge features de rede para source accounts
df_with_network = df_with_network.merge(
    network_features_df[['node', 'degree', 'in_degree', 'out_degree',
                        'degree_centrality', 'in_degree_centrality', 'out_degree_centrality']],
    left_on='source',
    right_on='node',
    how='left'
)

# Renomear colunas para source
df_with_network = df_with_network.rename(columns={
    'degree': 'source_degree',
    'in_degree': 'source_in_degree',
    'out_degree': 'source_out_degree',
    'degree_centrality': 'source_degree_centrality',
    'in_degree_centrality': 'source_in_degree_centrality',
    'out_degree_centrality': 'source_out_degree_centrality'
})

df_with_network = df_with_network.drop('node', axis=1, errors='ignore')

# Merge features de rede para target accounts
df_with_network = df_with_network.merge(
    network_features_df[['node', 'degree', 'in_degree', 'out_degree',
                        'degree_centrality', 'in_degree_centrality', 'out_degree_centrality']],
    left_on='target',
    right_on='node',
    how='left'
)

# Renomear colunas para target
df_with_network = df_with_network.rename(columns={
    'degree': 'target_degree',
    'in_degree': 'target_in_degree',
    'out_degree': 'target_out_degree',
    'degree_centrality': 'target_degree_centrality',
    'in_degree_centrality': 'target_in_degree_centrality',
    'out_degree_centrality': 'target_out_degree_centrality'
})

df_with_network = df_with_network.drop('node', axis=1, errors='ignore')

# Fill NaN values para features de rede
network_cols = [col for col in df_with_network.columns if col.startswith(('source_', 'target_')) and
                ('degree' in col or 'centrality' in col)]

for col in network_cols:
    df_with_network[col] = df_with_network[col].fillna(0)

# Amostra para análise IV
df_network_sample = df_with_network.sample(n=min(100000, len(df_with_network)), random_state=42)

In [12]:
# Calcular Information Value para features de rede
numeric_network_cols = [col for col in network_cols if col in df_network_sample.select_dtypes(include=[np.number]).columns]

iv_results = calculate_iv(
    df_network_sample,
    target_col='is_fraud',
    bins=10,
    max_iv=10.0,
    min_samples=1,
    max_unique_values=100000
)

network_iv_results = iv_results[iv_results['variable'].isin(numeric_network_cols)].copy()

predictive_features = get_predictive_features(network_iv_results, min_iv=0.02, exclude_suspect=True)

network_iv_results.to_csv('../artifacts/network_features_iv_analysis.csv', index=False)

Error calculating IV for source_degree: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error calculating IV for target_degree: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error calculating IV for source_degree: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Error calculating IV for target_degree: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().


In [13]:
# Ranking das features de rede por IV
network_iv_results.to_csv('../artifacts/network_features_iv_analysis.csv', index=False)

predictive_count = len(network_iv_results[network_iv_results['IV'] >= 0.02])
if predictive_count == 0:
    print("DECISÃO: EXCLUIR todas as features de rede")
else:
    print(f"DECISÃO: Usar {predictive_count} features de rede com IV >= 0.02")

DECISÃO: EXCLUIR todas as features de rede


## ▸ Features Categóricas

Aplicamos encoding seguro para variáveis categóricas, garantindo que não haja data leakage através de informações futuras.

In [14]:
# Preparar dados para encoding categórico usando função refatorada
df_categorical = df_clean.copy()

# Usar função importada para encoding
df_encoded, encoders = encode_categorical_features(df_categorical, target_col='is_fraud')

print(f"Features categóricas processadas: {len(encoders)} colunas encoded")
print(f"Dataset após encoding: {len(df_encoded.select_dtypes(include=[np.number]).columns)} colunas numéricas")

2025-10-18 15:20:12,612 - INFO - Encoding categorical features...
2025-10-18 15:21:02,399 - INFO - Encoded 14 categorical columns


Features categóricas processadas: 14 colunas encoded
Dataset após encoding: 22 colunas numéricas


## ▸ Validação de Features

Analisamos correlações, distribuição e possíveis problemas de leakage antes de criar o pipeline final.

In [ ]:
# Combinar todas as features criadas
print("Combinando features para validação...")

# Base: usar df_raw que tem todas as colunas originais necessárias para patterns
df_final = df_raw.copy()

# Mapear colunas para o formato esperado pela função de patterns
column_mapping = {
    'From Bank ID': 'from_bank',
    'To Bank ID': 'to_bank',
    'Amount Received': 'amount_received',
    'Receiving Currency': 'receiving_currency',
    'Payment Currency': 'payment_currency'
}
df_final = df_final.rename(columns=column_mapping)

print("Features temporais e de rede excluídas conforme análise de IV.")
print(f"Dataset final: {df_final.shape[0]:,} linhas × {df_final.shape[1]} colunas")

# Aplicar features baseadas em patterns
print("\nCriando features baseadas em patterns...")

# Inicializar Pattern Feature Engineer
pattern_engineer = PatternFeatureEngineer()

df_with_patterns = pattern_engineer.create_pattern_similarity_features(df_final.copy())

# Mostrar novas features criadas
new_pattern_features = set(df_with_patterns.columns) - set(df_final.columns)
print(f"Criadas {len(new_pattern_features)} novas features baseadas em patterns. Ex: {sorted(list(new_pattern_features))[:3]}...")

# Recalcular correlações incluindo pattern features
numeric_cols_updated = df_with_patterns.select_dtypes(include=[np.number]).columns
correlations_updated = df_with_patterns[numeric_cols_updated].corr()['is_fraud'].abs().sort_values(ascending=False)

print("Top 10 features mais correlacionadas (incluindo patterns):")
top_10 = correlations_updated.head(10)
for feature, corr in top_10.items():
    print(f"   • {feature}: {corr:.4f}")

# Salvar dataset atualizado
output_dir = Path('../data/processed/')
output_dir.mkdir(parents=True, exist_ok=True)
df_with_patterns.to_pickle(output_dir / 'features_with_patterns.pkl')
print("Dataset com patterns salvo!")

print("Features de patterns integradas com sucesso!")

# Salvar artefatos para reprodutibilidade

Combinando features para validação...

Features temporais e de rede excluídas conforme análise de IV.
Dataset final: 5,078,345 linhas × 31 colunas

Criando features baseadas em patterns...
Features temporais e de rede excluídas conforme análise de IV.
Dataset final: 5,078,345 linhas × 31 colunas

Criando features baseadas em patterns...
Loaded 3209 pattern transactions from 370 laundering attempts
Loaded 3209 pattern transactions from 370 laundering attempts


KeyError: 'from_bank'

In [24]:
# DEBUG: Verificar colunas disponíveis no df_raw
print("Colunas disponíveis no df_raw:")
print(f"Total de colunas: {len(df_raw.columns)}")
print(f"'from_bank' existe: {'from_bank' in df_raw.columns}")
print(f"'From Bank' existe: {'From Bank' in df_raw.columns}")
print(f"'From Bank ID' existe: {'From Bank ID' in df_raw.columns}")
print(f"'to_bank' existe: {'to_bank' in df_raw.columns}")
print(f"'To Bank' existe: {'To Bank' in df_raw.columns}")
print(f"'To Bank ID' existe: {'To Bank ID' in df_raw.columns}")

# Mostrar primeiras colunas
print(f"\nPrimeiras 10 colunas: {df_raw.columns.tolist()[:10]}")

Colunas disponíveis no df_raw:
Total de colunas: 31
'from_bank' existe: False
'From Bank' existe: False
Total de colunas: 31
'from_bank' existe: False
'From Bank' existe: False
'From Bank ID' existe: True
'to_bank' existe: False
'To Bank' existe: False
'From Bank ID' existe: True
'to_bank' existe: False
'To Bank' existe: False
'To Bank ID' existe: True

Primeiras 10 colunas: ['timestamp', 'From Bank ID', 'source', 'To Bank ID', 'target', 'Amount Received', 'Receiving Currency', 'amount', 'Payment Currency', 'payment_format']
'To Bank ID' existe: True

Primeiras 10 colunas: ['timestamp', 'From Bank ID', 'source', 'To Bank ID', 'target', 'Amount Received', 'Receiving Currency', 'amount', 'Payment Currency', 'payment_format']

